In [1]:
import streamlit as st
import base64
import os
from ultralytics import YOLO
import joblib
import pandas as pd
import torch

In [3]:
import torch.nn as nn

# Define or import the missing `_loss` class/module
class _loss(nn.Module):
    def __init__(self):
        super(_loss, self).__init__()

    def forward(self, input, target):
        # Example custom loss computation
        return ((input - target) ** 2).mean()

In [4]:
# we import model
cost_model = joblib.load("Models/cost_model.pkl")

ModuleNotFoundError: No module named '_loss'

In [ ]:




# Display the selected workshop
if buttom_workshop != "Select among available workshops":
    # workshop selection
    workshops={
    'Smithfield Autotech':'Low',
    'Sandyford McCann Motor':'Medium',
    'Mobile Mechanic':'High'
    }

    selected_workshop_qual = workshops[buttom_workshop]

    # we predict the cost of the claims
    case = pd.DataFrame({
    'const': [1],
    'brand_Volkswagen': [0],
    'model_Corolla': [0],
    'model_Golf': [0],
    'model_Polo': [0],
    'model_Tiguan': [0],
    'model_Yaris': [1],
    'veh_age_range_Newer': [1],
    'veh_age_range_Old': [0],
    'workshop_quality_Low': [0],
    'workshop_quality_Medium': [0],
    'counties_group2': [0],
    'counties_group3': [0],
    'damage_type_met_dent_medium': [0],
    'damage_type_met_dent_minor': [0],
    'damage_type_met_dent_severe': [0],
    'damage_type_met_tear': [0],
    'damage_type_mis_lamp': [0],
    'damage_type_mis_lost': [0],
    'damage_type_mis_punct': [0]
    })

    # Adjust case DataFrame based on selected_workshop_qual
    if selected_workshop_qual == 'High':
        case['workshop_quality_Low'] = [0]
        case['workshop_quality_Medium'] = [0]
    elif selected_workshop_qual == 'Medium':
        case['workshop_quality_Low'] = [0]
        case['workshop_quality_Medium'] = [1]
    elif selected_workshop_qual == 'Low':
        case['workshop_quality_Low'] = [1]
        case['workshop_quality_Medium'] = [0]

    damage_types = [results[0].names[int(cls)] for cls in results[0].boxes.cpu().numpy().cls]

    # Function to generate the complete_case DataFrame
    def generate_complete_case(case, damage_types):
        complete_case = pd.DataFrame()

        for damage_type in damage_types:
            temp_case = case.copy()

            # Set the corresponding damage type to 1
            damage_column = f'damage_type_{damage_type}'
            if damage_column in temp_case.columns:
                temp_case[damage_column] = [1]

            # Append this row to the complete_case DataFrame
            complete_case = pd.concat([complete_case, temp_case], ignore_index=True)

        return complete_case


    # Generate the complete_case DataFrame
    complete_case = generate_complete_case(case, damage_types)

    # dataframe with the predictions
    cost_predictions= pd.DataFrame({'Damages': damage_types,
    'Cost Estimates': cost_model.predict(complete_case).round(0)
    })

    # Calculate the total cost as the sum of all 'Cost Estimates'
    total_cost = cost_predictions['Cost Estimates'].sum()

    # Format total_cost as a string with no decimals and a comma as a thousands separator
    formatted_total_cost = f"{total_cost:,.0f}"